In [46]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Dev\amo-personal-project`

In [47]:
using JuMP, HiGHS, Plots, DataFrames

In [48]:
# Define all sets
T = collect(1:3) # time periods
Ω = collect(1:4) # scenarios

n_N = 24 # number of nodes
N = collect(1:n_N) # nodes
L = collect(1:34) # set of transmission lines
G_c = collect(1:12) # convetional generators
G_r = collect(1:4) # renewable generators

4-element Vector{Int64}:
 1
 2
 3
 4

In [49]:
# scenario probabilities
π_ω = [0.25,0.25,0.25,0.25]

4-element Vector{Float64}:
 0.25
 0.25
 0.25
 0.25

In [50]:
# Define connections between nodes as an upper triangular matrix
orig_n = [1, 1, 1, 2, 2, 3,  3, 4,  5,  6, 7, 8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 17, 17, 18, 19, 20, 21]
dest_n = [2, 3, 5, 4, 6, 9, 24, 9, 10, 10, 8, 9, 10, 11, 12, 11, 12, 13, 14, 13, 23, 23, 16, 16, 21, 24, 17, 19, 18, 22, 21, 20, 23, 22]

lines = zeros(Int8, n_N, n_N)

for i in 1:size(orig_n)[1]
    lines[orig_n[i], dest_n[i]] = i # incoming transmission lines
    lines[dest_n[i], orig_n[i]] = -i # outgoing transmission lines
end
lines

24×24 Matrix{Int8}:
  0   1   2   0   3    0    0    0   0  …    0    0    0    0    0   0   0  0
 -1   0   0   4   0    5    0    0   0       0    0    0    0    0   0   0  0
 -2   0   0   0   0    0    0    0   6       0    0    0    0    0   0   0  7
  0  -4   0   0   0    0    0    0   8       0    0    0    0    0   0   0  0
 -3   0   0   0   0    0    0    0   0       0    0    0    0    0   0   0  0
  0  -5   0   0   0    0    0    0   0  …    0    0    0    0    0   0   0  0
  0   0   0   0   0    0    0   11   0       0    0    0    0    0   0   0  0
  0   0   0   0   0    0  -11    0  12       0    0    0    0    0   0   0  0
  0   0  -6  -8   0    0    0  -12   0       0    0    0    0    0   0   0  0
  0   0   0   0  -9  -10    0  -13   0       0    0    0    0    0   0   0  0
  ⋮                    ⋮                ⋱                        ⋮          
  0   0   0   0   0    0    0    0   0  …   27    0   28    0    0   0   0  0
  0   0   0   0   0    0    0    0   0       

In [51]:
# demand 
D = [84, 75, 139, 58, 55, 106, 97, 132, 135, 150, 0, 0, 205, 150, 245, 77, 0, 258, 141, 100, 0, 0, 0, 0]

# transmission line capacities
F_max = [175, 175, 350, 175, 175, 175, 400, 175, 350, 175, 350, 175, 175, 400, 400, 400, 400,
        500, 500, 500, 500, 250, 250, 500, 400, 500, 500, 500, 500, 500, 1000, 1000, 1000, 500]

34-element Vector{Int64}:
  175
  175
  350
  175
  175
  175
  400
  175
  350
  175
    ⋮
  500
  500
  500
  500
  500
 1000
 1000
 1000
  500

In [52]:
# cost of generation for convetional generators
c = [13.32, 13.32, 20.7, 20.93, 26.11, 10.52, 10.52, 6.02, 5.47, 7, 10.52, 10.89]
# c = [20.93, 26.11, 10.52, 10.52, 6.02, 5.47, 7, 10.52, 10.89]

# Location of the generators
loc_G_c = [1, 2, 7, 13, 15, 15, 16, 18, 21, 22, 23, 23]
# loc_G_c = [13, 15, 15, 16, 18, 21, 22, 23, 23]
loc_G_r = [3, 5, 16, 21]
# loc_G_r = [1, 2, 7, 3, 5, 16, 21]

# maximum generation capacity of convetional generators
P_c_max = [106.4, 106.4, 245, 413.7, 42, 108.5, 108.5, 280, 280, 210, 217, 245] 
# P_c_max = [413.7, 42, 108.5, 108.5, 280, 280, 210, 217, 245] 

# maximum generation capacity of renewable generators
P_r_max = [
    [500, 500, 500, 500], # t=1
    [150, 150, 300, 300], # t=2
    [50, 100, 200, 450]   # t=3
]

3-element Vector{Vector{Int64}}:
 [500, 500, 500, 500]
 [150, 150, 300, 300]
 [50, 100, 200, 450]

In [53]:
println("Total demand: " * string(sum(D)))
println("Maximum supply of convetional generators: " * string(sum(P_c_max)))
println("Maximum supply of renewable generators: " * string(size(G_r)[1]*500))

Total demand: 2207
Maximum supply of convetional generators: 2362.5
Maximum supply of renewable generators: 2000


In [54]:
# battery capacity
B_max = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

24-element Vector{Int64}:
 100
 100
 100
 100
 100
 100
 100
 100
 100
 100
   ⋮
 100
 100
 100
 100
 100
 100
 100
 100
 100

## Define and solve model

In [55]:
function get_lines_for_node(node, incoming)
    if incoming
        return filter(i -> i >= 1.0, lines[node, 1:end])
    end
    return filter(i -> i <= -1.0, lines[node, 1:end])
end

function get_generator_for_node(node, loc_G)
    return findall(i -> i == node, loc_G)
end

get_generator_for_node (generic function with 1 method)

In [56]:
model = Model(HiGHS.Optimizer)

@variable(model, P_c_max[g] >= P_c[g in G_c, t in T, ω in Ω] >= 0) 
@variable(model, P_r_max[t][ω] >= P_r[g in G_r, t in T, ω in Ω] >= 0) 
@variable(model, F_max[l] >= F[l in L, t in T, ω in Ω] >= -F_max[l])
@variable(model, D[n] >= L_shed[n in N] >= 0)

@variable(model, B_max[n] >= B[n in N, t in vcat([0],T), ω in Ω] >= 0) 
@variable(model, ΔB[n in N, t in T, ω in Ω])

@constraint(model, init_battery_charge[n in N, ω in Ω], B[n,0,ω] == 0)
@constraint(model, battery_charge[n in N, t in T, ω in Ω], B[n,t-1,ω] + ΔB[n,t,ω] == B[n,t,ω])

@constraint(model, power_flow[n in N, t in T, ω in Ω], 
    sum(P_c[g,t,ω] for g in get_generator_for_node(n, loc_G_c))
    + sum(P_r[g,t,ω] for g in get_generator_for_node(n, loc_G_r))
    - D[n]
    + L_shed[n]
    + sum(F[i,t,ω] for i in get_lines_for_node(n, true))
    - sum(F[-i,t,ω] for i in get_lines_for_node(n, false))
    - ΔB[n,t,ω]
    == 0
)

for g in G_r
    for ω in Ω[2:end]
        @constraint(model, P_r[g,1,ω-1]==P_r[g,1,ω])
    end

    @constraint(model, P_r[g,2,1]==P_r[g,2,2])
    @constraint(model, P_r[g,2,3]==P_r[g,2,4])
end

@objective(model, Min, sum(π_ω[ω]*c[g]*P_c[g,t,ω] for g in G_c, t in T, ω in Ω) + 100000*sum(L_shed[n] for n in N))

3.33 P_c[1,1,1] + 3.33 P_c[1,1,2] + 3.33 P_c[1,1,3] + 3.33 P_c[1,1,4] + 3.33 P_c[1,2,1] + 3.33 P_c[1,2,2] + 3.33 P_c[1,2,3] + 3.33 P_c[1,2,4] + 3.33 P_c[1,3,1] + 3.33 P_c[1,3,2] + 3.33 P_c[1,3,3] + 3.33 P_c[1,3,4] + 3.33 P_c[2,1,1] + 3.33 P_c[2,1,2] + 3.33 P_c[2,1,3] + 3.33 P_c[2,1,4] + 3.33 P_c[2,2,1] + 3.33 P_c[2,2,2] + 3.33 P_c[2,2,3] + 3.33 P_c[2,2,4] + 3.33 P_c[2,3,1] + 3.33 P_c[2,3,2] + 3.33 P_c[2,3,3] + 3.33 P_c[2,3,4] + 5.175 P_c[3,1,1] + 5.175 P_c[3,1,2] + 5.175 P_c[3,1,3] + 5.175 P_c[3,1,4] + 5.175 P_c[3,2,1] + 5.175 P_c[3,2,2] + 5.175 P_c[3,2,3] + 5.175 P_c[3,2,4] + 5.175 P_c[3,3,1] + 5.175 P_c[3,3,2] + 5.175 P_c[3,3,3] + 5.175 P_c[3,3,4] + 5.2325 P_c[4,1,1] + 5.2325 P_c[4,1,2] + 5.2325 P_c[4,1,3] + 5.2325 P_c[4,1,4] + 5.2325 P_c[4,2,1] + 5.2325 P_c[4,2,2] + 5.2325 P_c[4,2,3] + 5.2325 P_c[4,2,4] + 5.2325 P_c[4,3,1] + 5.2325 P_c[4,3,2] + 5.2325 P_c[4,3,3] + 5.2325 P_c[4,3,4] + 6.5275 P_c[5,1,1] + 6.5275 P_c[5,1,2] + 6.5275 P_c[5,1,3] + 6.5275 P_c[5,1,4] + 6.5275 P_c[5,2,1] + 

In [57]:
optimize!(model)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
480 rows, 1077 cols, 2076 nonzeros
456 rows, 1053 cols, 2028 nonzeros
Presolve : Reductions: rows 456(-236); columns 1053(-243); elements 2028(-556)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.2342224920e-02 Pr: 264(112596); Du: 0(6.21502e-11) 0s
        594     0.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 594
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.01


In [58]:
objective_value(model)

0.0

## View results

In [59]:
results = Dict(
    "P_c" => value.(P_c),
    "P_r" => value.(P_r),
    "F" => value.(F),
    "B" => value.(B),
    "ΔB" => value.(ΔB),
    "objective" => objective_value(model)
)

Dict{String, Any} with 6 entries:
  "ΔB"        => 3-dimensional DenseAxisArray{Float64,3,...} with index sets:…
  "B"         => 3-dimensional DenseAxisArray{Float64,3,...} with index sets:…
  "P_r"       => 3-dimensional DenseAxisArray{Float64,3,...} with index sets:…
  "objective" => 0.0
  "P_c"       => 3-dimensional DenseAxisArray{Float64,3,...} with index sets:…
  "F"         => 3-dimensional DenseAxisArray{Float64,3,...} with index sets:…

In [61]:
print(value.(P_c))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    Dimension 2, [1, 2, 3]
    Dimension 3, [1, 2, 3, 4]
And data, a 12×3×4 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [62]:
print(value.(P_r))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
    Dimension 2, [1, 2, 3]
    Dimension 3, [1, 2, 3, 4]
And data, a 4×3×4 Array{Float64, 3}:
[:, :, 1] =
 -0.0  -0.0  0.0
 -0.0  -0.0  0.0
 -0.0  -0.0  0.0
 -0.0  -0.0  0.0

[:, :, 2] =
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0
 -0.0  0.0  0.0

[:, :, 3] =
  0.0  -0.0  0.0
  0.0  -0.0  0.0
  0.0  -0.0  0.0
 -0.0  -0.0  0.0

[:, :, 4] =
 -0.0  0.0   0.0
 -0.0  0.0   0.0
 -0.0  0.0  -0.0
 -0.0  0.0   0.0

In [63]:
print(value.(B))

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    Dimension 2, [0, 1, 2, 3]
    Dimension 3, [1, 2, 3, 4]
And data, a 24×4×4 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0

In [64]:
value.(ΔB)[:,:,4]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    Dimension 2, [1, 2, 3]
And data, a 24×3 Matrix{Float64}:
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
  ⋮          
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0